In [10]:
# Data Cleansing

import pandas as pd
import re
import matplotlib as plt

cr_df = pd.read_json('./complete_data.json')

def convert_cost(cost_str):
    import difflib
    
    s = cost_str.lower()
    if s != '-' and s != 'tbd':
        s = re.sub('[a-zA-Z]','',s)
        s = s.replace(",","")
        indicator = '.'
        if indicator in s:
            cost = re.findall("\d+\.\d+", s)
        else:
            cost = re.findall("[0-9]+", s)
    else:
        cost = s
        
    if cost_str.endswith("M") or cost_str.endswith("m"):
        return float(cost[0]) * 1000000
 
    end = re.findall('[a-zA-Z]+'+r'on\b', cost_str)
    measures = {'billion':1000000000,'million':1000000,'m':1000000}
    if cost in ['tbd', '-'] or len(cost) == 0:
        return None
    elif len(end) > 0:
        num = cost[0]
        fixed_measure = difflib.get_close_matches(end[0].lower(), ['billion','million','m'])[0]
        return float(num) * measures[fixed_measure]
    else:
        num = ''.join(cost).replace(',','')
        return float(num)
    
def get_currency(cost):
    if cost in ["TBD", "Not Indicated"]:
        return cost
    
    import difflib
    cost = cost.lower()
    cost = re.sub('\d+[,.]*', '', cost)
    word = cost.split()[0]
    if word not in ['tbd','-','not']:
        if len(difflib.get_close_matches(word, ['us','$','php'])) != 0:
            curr = {'us':'USD','$':'USD','php':'PHP'}
            return curr[difflib.get_close_matches(word, ['us','$','php'])[0]]
        else:
            return None
    else:
        return None

def get_cost(data):
    currency = data['Currency']
    
    if currency in ["TBD", "Not Indicated"]:
        return currency
    
    numerical = data['Numerical Cost']
    
    if numerical == None or pd.isnull(numerical):
        return '-'
    
    if currency == 'USD':
        numerical *= 50.54  

    numerical = f"{numerical:,.2f}"
    return 'PHP ' + numerical
        
    
# Debugger
'''for i,s in enumerate(cr_df['Cost']):
    print(f'\nIndex {i}')
    try:
        convert_cost(s)
        get_currency(s)
    except:
        print(f'Error')'''

cr_df['Numerical Cost'] = cr_df['Cost'].apply(convert_cost)
cr_df['Currency'] = cr_df['Cost'].apply(get_currency)

cr_df['Cost (PHP)'] = cr_df[['Numerical Cost', 'Currency']].apply(get_cost, axis=1)

pd.set_option('display.max_rows', None)
display(cr_df)

,Project Title,Region,Sector,Status,Award,Implementing Agency,Cost,Numerical Cost,Currency,Cost (PHP)
0,PPP for School Infrastructure Project Phase II...,"REGION X - NORTHERN MINDANAO,CARAGA",Education,Under Construction,Awarded,Department of Education (DepEd),"Php 1,604.59 Million",1.604590e+09,PHP,"PHP 1,604,590,000.00"
1,PPP for School Infrastructure Project Phase I ...,REGION IV-A - CALABARZON,Education,Operational,Awarded,Department of Education (DepEd),"Php 7,604.90 Million",7.604900e+09,PHP,"PHP 7,604,900,000.00"
2,PPP for School Infrastructure Project Phase I ...,REGION III - CENTRAL LUZON,Education,Operational,Awarded,Department of Education (DepEd),"Php 5,229.90 Million",5.229900e+09,PHP,"PHP 5,229,900,000.00"
3,Tri-City Ferry System Project,REGION I - ILOCOS,"Transportation,Ferry",Projects under Development,None,Local Governments of Alaminos City and Dagupa...,TBD,NaN,TBD,TBD
4,Metro Manila Skyway (Stage 2),NATIONAL CAPITAL REGION,Road,Operational,Awarded,Philippine National Construction Corporation (...,TBD,NaN,TBD,TBD
5,San Pablo City Water District Septage Manageme...,REGION IV-A - CALABARZON,"Environmental,Sanitation",Projects under Procurement,None,San Pablo City Water District,"Php 110,817,774.31",1.108178e+08,PHP,"PHP 110,817,774.31"
6,MWSS Privatization Project (East),NATIONAL CAPITAL REGION,Water and Sanitation,Operational,Awarded,Metropolitan Waterworks and Sewerage System (M...,-,NaN,None,-
7,Joint Venture Agreement for the Development an...,NATIONAL CAPITAL REGION,Property Development,Unknown,Awarded,National Housing Authority,Php 6.128 Billion,6.128000e+09,PHP,"PHP 6,128,000,000.00"
8,Modernization of Cebu Carbon Market,REGION VII - CENTRAL VISAYAS,"Agriculture,Agriculture/Food Security",Under Pre-Construction,Awarded,City Government of Cebu,Php 5.5 Billion,5.500000e+09,PHP,"PHP 5,500,000,000.00"
9,Municipality of Malay Ecological Solid Waste M...,REGION VI - WESTERN VISAYAS,"Environmental,Solid Waste Management",Operational,Awarded,Municipal Government of Malay,-,NaN,None,-


In [69]:
total_costs=pd.DataFrame(cr_df['Numerical Cost'].sum(), index = ["Total Costs for all Projects"], columns=["Numerical Cost"])

total_costs

,Numerical Cost
Total Costs for all Projects,2.768842e+13


In [75]:
status = list(set(cr_df['Status']))

status_table=[]
for i in status:
    status_table.append(pd.DataFrame(cr_df.loc[(cr_df['Status'] == i), 'Numerical Cost'].sum(), index = [i], columns=["Total Cost"]))

result=status_table[0]
for i in range(1,len(status_table)):
    result=result.append(status_table[i])

final_table=result.sort_values(by=['Total Cost'], ascending=False)
    
display(final_table)

,Total Cost
Operational,2.469276e+13
For Approval of Relevant Government Bodies,1.828658e+12
Under Pre-Construction,7.839613e+11
Under Construction,3.067808e+11
Projects under Development,3.142935e+10
Unsolicited Projects Under Competitive Challenge,2.210000e+10
Completed/Concluded,8.099310e+09
Unsolicited Proposal Under Initial Evaluation by Implementing Agency,7.448500e+09
Unknown,6.128000e+09
Projects under Procurement,1.050818e+09


In [76]:
sectors = list(set(cr_df['Sector']))

sectors_table=[]
for i in sectors:
    sectors_table.append(pd.DataFrame(cr_df.loc[(cr_df['Sector'] == i), 'Numerical Cost'].sum(), index = [i], columns=["Total Cost"]))

result=sectors_table[0]
for i in range(1,len(sectors_table)):
    result=result.append(sectors_table[i])

final_table=result.sort_values(by=['Total Cost'], ascending=False)
    
display(final_table)

,Total Cost
Road,2.476590e+13
"Transportation,Airport",1.037876e+12
"Transportation,Rail",8.085045e+11
"Land,Road",5.360300e+11
Property Development,2.093371e+11
"Transportation,Port",9.903600e+10
Water & Sanitation,3.633676e+10
"Environmental,Water,Water and Sanitation",3.134749e+10
Solid Waste Management,2.257100e+10
Hospitals & Healthcare,2.225800e+10


In [77]:
sectors = list(set(cr_df['Sector']))

sectors_table=[]
for i in sectors:
    sectors_table.append(pd.DataFrame(cr_df.loc[(cr_df['Status'] == 'Operational') & (cr_df['Sector'] == i), 
                                                'Numerical Cost'].sum(), index = [i], columns=["Total Cost"]))

result=sectors_table[0]
for i in range(1,len(sectors_table)):
    result=result.append(sectors_table[i])
    
final_table=result.sort_values(by=['Total Cost'], ascending=False)
    
display(final_table)

,Total Cost
Road,2.454466e+13
Water & Sanitation,3.623176e+10
"Environmental,Water,Water and Sanitation",2.819380e+10
Education,1.628070e+10
Property Development,1.617854e+10
"Transportation,Airport",1.505600e+10
Airport,5.610000e+09
Information Technology,5.201600e+09
Power,5.037000e+09
"Environmental,Water and Sanitation",4.967230e+09


In [78]:
sectors = list(set(cr_df['Sector']))

sectors_table=[]
for i in sectors:
    sectors_table.append(pd.DataFrame(cr_df.loc[(cr_df['Status'] == 'Completed/Concluded') & (cr_df['Sector'] == i), 
                                                'Numerical Cost'].sum(), index = [i], columns=["Total Cost"]))

result=sectors_table[0]
for i in range(1,len(sectors_table)):
    result=result.append(sectors_table[i])
    
final_table=result.sort_values(by=['Total Cost'], ascending=False)
    
display(final_table)

,Total Cost
Power,4.377360e+09
Information Technology,3.506000e+09
Property Development,1.559500e+08
Energy,6.000000e+07
"Environmental,Sanitation",0.000000e+00
Airport,0.000000e+00
"Environmental,Water,Water and Sanitation",0.000000e+00
"Transportation,Rail",0.000000e+00
Solid Waste Management,0.000000e+00
Water,0.000000e+00


In [79]:
regions = list(set(cr_df['Region']))

regions_table=[]
for i in regions:
    regions_table.append(pd.DataFrame(cr_df.loc[(cr_df['Region'] == i), 
                                                'Numerical Cost'].sum(), index = [i], columns=["Total Cost"]))

result=regions_table[0]
for i in range(1,len(regions_table)):
    result=result.append(regions_table[i])
    
final_table=result.sort_values(by=['Total Cost'], ascending=False)
    
display(final_table)

,Total Cost
"REGION I - ILOCOS,REGION III - CENTRAL LUZON",2.441519e+13
NATIONAL CAPITAL REGION,1.230677e+12
REGION III - CENTRAL LUZON,8.860068e+11
"REGION IV-A - CALABARZON,NATIONAL CAPITAL REGION",3.248950e+11
,2.048400e+11
REGION IV-A - CALABARZON,1.467917e+11
REGION VII - CENTRAL VISAYAS,1.339844e+11
REGION XI - DAVAO,1.137441e+11
"REGION III - CENTRAL LUZON,NATIONAL CAPITAL REGION",1.005645e+11
REGION VI - WESTERN VISAYAS,5.067430e+10


In [ ]:
# Data Sorting

statuses = cr_df.Status.unique()

for status in statuses:
    df = cr_df.loc[cr_df["Status"] == status]
    sectors = df['Sector'].value_counts().index.tolist()

    new_df = pd.DataFrame(columns=list(df.columns))
    for sector in sectors:
        projects = df.loc[df["Sector"] == sector]
        projects = projects.sort_values('Numerical Cost', ascending=False)
        new_df = pd.concat([new_df, projects])
    
    final_df = new_df.filter(["Sector", "Project Title", "Implementing Agency", "Cost (PHP)", "Region", "Award"], axis=1)
    
    final_df = final_df.style.set_caption(status).set_table_styles([{
            'selector': 'caption',
            'props': [
                ('color', 'black'),
                ('font-size', '24px')
            ]
        }])
    final_df = final_df.hide_index()
    
    display(final_df)

In [ ]:
import openpyxl

final_df.to_excel("output.xlsx")